#### Secret Types

You can use the `SecretStr` and the `SecretBytes` data types for storing sensitive information that you do not want to be visible in logging or tracebacks. `SecretStr` and `SecretBytes` can be initialized idempotently or by using `str` or `bytes` literals respectively. The `SecretStr` and `SecretBytes` will be formatted as either `'**********'` or `''` on conversion to json.

In [1]:
from pydantic import BaseModel, SecretStr, SecretBytes, ValidationError

In [2]:
class Model(BaseModel):
    password: SecretStr
    password_bytes: SecretBytes

In [3]:
sm = Model(password="IAmSensitive", password_bytes=b"IAmSensitiveBytes")
print(f"{sm = }")
print(f"{sm.password = }")
print(f"{sm.dict() = }")
print(f"{sm.json() = }")

sm = Model(password=SecretStr('**********'), password_bytes=SecretBytes(b'**********'))
sm.password = SecretStr('**********')
sm.dict() = {'password': SecretStr('**********'), 'password_bytes': SecretBytes(b'**********')}
sm.json() = '{"password": "**********", "password_bytes": "**********"}'


In [4]:
print(f"{sm.password.get_secret_value() = }")
print(f"{sm.password_bytes.get_secret_value() = }")

sm.password.get_secret_value() = 'IAmSensitive'
sm.password_bytes.get_secret_value() = b'IAmSensitiveBytes'


In [5]:
try:
    m = Model(password=[1, 2, 3], password_bytes=[1, 2, 3])
    print(m)
except ValidationError as e:
    print(e)

2 validation errors for Model
password
  str type expected (type=type_error.str)
password_bytes
  byte type expected (type=type_error.bytes)


In [6]:
class ModelDumpable(BaseModel):
    password: SecretStr
    password_bytes: SecretBytes
    
    class Config:
        json_encoders = {
            SecretStr: lambda v: v.get_secret_value() if v else None,
            SecretBytes: lambda v: v.get_secret_value() if v else None,
        }

In [7]:
sm = ModelDumpable(password="IAmSensitive", password_bytes=b"IAmSensitiveBytes")
print(f"{sm = }")
print(f"{sm.password = }")
print(f"{sm.dict() = }")
print(f"{sm.json() = }")

sm = ModelDumpable(password=SecretStr('**********'), password_bytes=SecretBytes(b'**********'))
sm.password = SecretStr('**********')
sm.dict() = {'password': SecretStr('**********'), 'password_bytes': SecretBytes(b'**********')}
sm.json() = '{"password": "IAmSensitive", "password_bytes": "IAmSensitiveBytes"}'
